In [1]:
#Importing unityEnvironment work simulation to be done here along with numpy

from unityagents import UnityEnvironment
import numpy as np

In [3]:
#Loading Unity Environment with the downloaded environment app files

env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [9]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

brain_name, brain

('ReacherBrain', <unityagents.brain.BrainParameters at 0x11f89ed68>)

In [13]:
env_info = env.reset(train_mode=True)[brain_name]
num_agents = env_info.agents

len(env_info.agents)

20

In [2]:
from ddpg_agent import Agent

agent = Agent()
agent

TypeError: __init__() missing 3 required positional arguments: 'state_size', 'action_size', and 'random_seed'

In [14]:
a = np.array([1,2,3,5,1,0,4,-1])

np.clip(a,-1,2)

array([ 1,  2,  2,  2,  1,  0,  2, -1])

In [20]:
states = env_info.vector_observations;states.shape

(20, 33)